# Web Scraping: Website Scrap

## Iswarya SIVASUBRAMANIAM DIA1

In [10]:
import bs4
from urllib import request
import pandas as pd
from itertools import zip_longest

In [2]:
#URL du site à scraper
url_ligue_1="https://www.fiches-auto.fr/articles-auto/voiture-electrique/s-852-comparatif-des-voitures-electriques.php"

In [3]:
request_text = request.urlopen(url_ligue_1).read()

In [4]:
htmlpage = bs4.BeautifulSoup(request_text, "html")

In [5]:
# création d'une  liste pour stocker les élements du site
td_elements=[]

In [6]:
tables = htmlpage.find_all('table')

In [7]:
# Nous scrapons les éléments de chaque cellules des tables du site internet
#Les 3 premières tables de notre site ne continennet pas les informations interessantes:
for index, table in enumerate(tables[:-3]):

    print(f"Table {index + 1}:") 
    rows = table.find_all('tr')
    is_first_row=True
    for row in rows:
        if is_first_row:
            is_first_row = False
            continue 
        columns = row.find_all(['th', 'td'])
        for col in columns:
            td_elements.append(col.text.strip())
            print(col.text.strip(), end='\t')
        print() 
    print()  

Table 1:
Ami (8 ch)	0Sec.	45km/h	0Dm3	5.5kWh	70km	

Table 2:
C-Zero (67 ch)	15.9Sec.	130km/h	166Dm3	14.5kWh	100km	
Spring (45 ch)	19.1Sec.	125km/h	270Dm3	26.8kWh	230km	
Spring (65 ch)	13.7Sec.	125km/h	270Dm3	26.8kWh	220km	
500e (95 ch)	9.5Sec.	135km/h	185Dm3	21.3kWh	190km	
500e (118 ch)	9Sec.	150km/h	185Dm3	37.3kWh	321km	
T03 (109 ch)	12Sec.	130km/h	210Dm3	41.3kWh	417km	
Leaf (109 ch)	11.5Sec.	144km/h	370Dm3	22kWh	199km (NEDC)	
Ion (67 ch)	15.9Sec.	130km/h	166Dm3	14.5kWh	100km	
	0 à 100	V max	Coffre	Bat.kWh	Autonomie	
Twingo III (82 ch)	12.9Sec.	135km/h	240Dm3	22kWh	190km	
Zoe (88 ch)	13.5Sec.	135km/h	338Dm3	22kWh	210km (NEDC)	
Zoe (77 ch)	15.5Sec.	135km/h	338Dm3	41kWh	400km (NEDC)	
Zoe (88 ch)	13.2Sec.	135km/h	338Dm3	41kWh	300km	
Zoe (92 ch)	13.2Sec.	135km/h	338Dm3	41kWh	317km	
Zoe (108 ch)	11.4Sec.	135km/h	338Dm3	41kWh	300km	
Mii EV (83 ch)	12.3Sec.	130km/h	251Dm3	32.3kWh	258km	
Fortwo (82 ch)	11.5Sec.	130km/h	260Dm3	17.6kWh	135km	
E-Up! (82 ch)	12.4Sec.	130km/h	250Dm3	18.7kWh	160km 

In [8]:
# cération d'un DataFrame pour stocker les élements du site
Modèle_voiture = []
Acceleration = []
Vmax = []
Coffre = []
Batterie = []
Autonomie = []
for i in range(0, len(td_elements), 6):  
    car_name = td_elements[i]
    Acceleration_value = td_elements[i + 1]
    Vmax_value = td_elements[i + 2]
    Coffre_value = td_elements[i + 3]
    Batterie_value = td_elements[i + 4]
    Autonomie_value = td_elements[i + 5]

    Modèle_voiture.append(car_name)
    Acceleration.append(Acceleration_value)
    Vmax.append(Vmax_value)
    Coffre.append(Coffre_value)
    Batterie.append(Batterie_value)
    Autonomie.append(Autonomie_value)

data = {
    'Car Name': Modèle_voiture,
    'Acceleration 0 à 100(Sec)': Acceleration,
    'Max Speed (km/h)': Vmax,
    'Cargo Volume (Dm3)': Coffre,
    'Battery (kWh)': Batterie,
    'Autonomie (km)': Autonomie,
}
df = pd.DataFrame(data)

In [9]:
df

,Car Name,Acceleration 0 à 100(Sec),Max Speed (km/h),Cargo Volume (Dm3),Battery (kWh),Autonomie (km)
0,Ami (8 ch),0Sec.,45km/h,0Dm3,5.5kWh,70km
1,C-Zero (67 ch),15.9Sec.,130km/h,166Dm3,14.5kWh,100km
2,Spring (45 ch),19.1Sec.,125km/h,270Dm3,26.8kWh,230km
3,Spring (65 ch),13.7Sec.,125km/h,270Dm3,26.8kWh,220km
4,500e (95 ch),9.5Sec.,135km/h,185Dm3,21.3kWh,190km
...,...,...,...,...,...,...
211,EQS (523 ch),4.3Sec.,210km/h,610Dm3,107.8kWh,651km
212,Taycan (625 ch),3.2Sec.,260km/h,366Dm3,83.7kWh,450km
213,Taycan (625 ch),2.8Sec.,260km/h,366Dm3,83.7kWh,412km
214,Taycan Cross Turismo (625 ch),3.3Sec.,250km/h,405Dm3,83.7kWh,452km


## Scraping from a PDF

In [4]:
import fitz
import requests

In [5]:

pdf_url = "http://johnhwman.free.fr/Divers/Comparatif_VE.pdf"

response = requests.get(pdf_url)
pdf_content = response.content

pdf_document = fitz.open("pdf", pdf_content)

for page_number in range(pdf_document.page_count):
    page = pdf_document[page_number]
    text = page.get_text()
    print(f"Page {page_number + 1}:\n{text}\n")

pdf_document.close()


Page 1:
Comparatif de l'efficacité énergétique de 161 modèles de véhicules électriques du marché 2019-2022
Les meilleurs rendements (< 134 Wh/km / cycle WLTP)
Marque / Modèle
Poids à Vide/Cx/Scx
Modes de charge / puissances
5,8kWh / 5,35kWh
12 ch / 9 kW / 40 Nm
N/D
45 km/h
471 kg / ? / ?
N/D
N/D
1,84kW AC (Prise domestique)
0,82€ @ 1,8kW
1,09 €
60kWh / 60kWh
136 ch / 100 kW / 1200 Nm
10 sec.
150 km/h
875-505 km / 69-119 Wh/km
565-405 km / 106-148 Wh/km
Type 2 - 22kW AC / CCS 75 kW DC 
8,85€ @ 22kW
1,18 €
23 kWh / 21,3 kWh
82 ch / 60 kW / 160 Nm
12,6 sec.
135 km/h
205-135 km / 104-158 Wh/km
120-90 km / 178-237 Wh/km
Type 2 Accélérée - 22kW AC
3,14€ @ 22kW
1,65 €
15 350 - 20 450 €
54kWh / 51kWh
325 ch / 239 kW / 420 Nm
5,6 sec.
225 km/h
515-330 km / 99-155 Wh/km
330-250 km / 155-204 Wh/km
Type 2 - 11kW AC / CCS 170kW DC
7,52€@11kW / 12,24€ (SUC)
1,67€/2,73€ (SUC)
42 kWh / 37,3 kWh
118 ch / 87 kW / 220 Nm
9 sec.
150 km/h
375-240 km / 99-155 Wh/km
225-175 km / 175-230 Wh/km
Type 2 - 11kW A

In [6]:
text

"125 kWh / 125 kWh\n435 ch / 320 kW / 320 Nm\n3,9 sec.\n250 km/h\nN/D\nN/D\n> 71 000 €\n120 kWh / 120 kWh\n? ch / ? kW / ? Nm\n4,5 sec.\n193 km/h\n660-480 km / 182-250 Wh/km\n405-325 km / 293-369 Wh/km\nType 2 - 11kW AC / CCS 250kW DC\n17,7€ @ 11kW / 28,8€ (SUC)\n3,85€ / 6,26€ (SUC)\n53000 € ?\n200 kWh / 200 kWh\n816 ch / 600 kW / 1400 Nm\n3 sec.\n209 km/h\n1080-780 km / 185-256 Wh/km\n670-540 km / 299-370 Wh/km\nType 2 - 11kW AC / CCS 250kW DC\n29,5€ @ 11kW / 48€ (SUC)\n3,96€ / 6,4€ (SUC)\n78000 € ?\nPage 2/3\nNotes et explications:\nLes lignes surlignées en jaune clair indiquent des modèles qui ne sont pas encore complètement spécifiés par leurs fabricants: ce sont donc des valeurs estimées. Un lien sur une image du véhicule est disponible en première colonne.\nConsommation (Wh/km) = Capacité de la batterie utile (kWh) / Autonomie annoncée (km) :\nNote : \nLe poids à vide du véhicule, la température ambiante, la trainée aérodynamique et le rendement de l’alimentation DC/DC de la batt

In [8]:
import fitz
import re
import requests

# Replace the URL with the actual URL of your PDF
pdf_url = "http://johnhwman.free.fr/Divers/Comparatif_VE.pdf"

# Download the PDF (you can use requests or any other library for this)
response = requests.get(pdf_url)
pdf_content = response.content

# Open the PDF using PyMuPDF
pdf_document = fitz.open("pdf", pdf_content)

# Loop through pages
for page_number in range(pdf_document.page_count):
    page = pdf_document[page_number]

    # Get the annotations on the page
    annotations = page.annots()

    # Extract and print the text associated with each link
    for annotation in annotations:
        if annotation.info.get("title", "") == "uri":
            link_text = annotation.info.get("subject", "")
            link_url = annotation.info.get("title", "")
            print("1")
            print(f"Page {page_number + 1} - Link Text: {link_text}, Link URL: {link_url}")

# Close the PDF document
pdf_document.close()


In [17]:
import requests
import bs4
import pandas as pd

url = "https://www.numerama.com/vroom/599805-voitures-electriques-tous-les-modeles-disponibles-en-france-en-2020.html"

response = requests.get(url)
html_content = response.text
soup = bs4.BeautifulSoup(html_content, 'html.parser')

# Find all the <h2> elements with class 'wp-block-heading'
modele_elements = soup.find_all('h2', class_='wp-block-heading')


modeles = []
puissances = []
autonomies = []
prix = []
bonus = []

for modele_element in modele_elements:
    modele = modele_element.text.strip()
    modeles.append(modele)
    print(modele)

    figure_element = modele_element.find_next('figure', class_='wp-block-table')
    
    if figure_element:
        puissance = figure_element.find('tr', {'td': 'Puissance'}).find('td', {'td': 'Puissance'}).text.strip()
        autonomie = figure_element.find('tr', {'td': 'Autonomie (WLTP)'}).find('td', {'td': 'Autonomie (WLTP)'}).text.strip()
        prix_value = figure_element.find('tr', {'td': 'Prix'}).find('td', {'td': 'Prix'}).text.strip()
        bonus_value = figure_element.find('tr', {'td': 'Bonus'}).find('td', {'td': 'Bonus'}).text.strip()

        puissances.append(puissance)
        autonomies.append(autonomie)
        prix.append(prix_value)
        bonus.append(bonus_value)
    else:
        # If no <figure> element is found, append None for missing data
        puissances.append(None)
        autonomies.append(None)
        prix.append(None)
        bonus.append(None)

df = pd.DataFrame({
    'Modele': modeles,
    'Puissance': puissances,
    'Autonomie': autonomies,
    'Prix': prix,
    'Bonus': bonus
})

print(df)


Prix d’une voiture électrique et bonus écologique


AttributeError: 'NoneType' object has no attribute 'find'